In [1]:
import os
import tensorflow  as tf
import time

In [2]:
def decode(serialized_example):
    alphabets=list()
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'cho_label':tf.FixedLenFeature([], tf.int64),
          'jung_label':tf.FixedLenFeature([], tf.int64),
          'jong_label':tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    cho_label=tf.cast(features['cho_label'],tf.int64)
    jung_label=tf.cast(features['jung_label'],tf.int64)
    jong_label=tf.cast(features['jong_label'],tf.int64)
    
    return image,label, cho_label, jung_label, jong_label

In [3]:
batch_size=100
test_file_name = "Shuffled_with_alphabet_PE92_Test.tfrecord"
train_file_name = "Shuffled_with_alphabet_PE92_Train.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

In [4]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [5]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_label_cho = tf.placeholder(tf.int64,[None])
input_label_jung = tf.placeholder(tf.int64,[None])
input_label_jong = tf.placeholder(tf.int64,[None])
input_labels= tf.placeholder(tf.int64,[None])


is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
print(input_images)


Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)


In [6]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=is_training)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [7]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=is_training)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=is_training)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [9]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=is_training)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [10]:
W_cho1 = tf.Variable(tf.random_normal([4*4*256, 256], stddev=0.01))
W_jung1 = tf.Variable(tf.random_normal([4*4*256, 256], stddev=0.01))
W_jong1 = tf.Variable(tf.random_normal([4*4*256, 256], stddev=0.01))
fc = tf.reshape(pool4,[-1,4*4*256])

fc_cho = tf.matmul(fc, W_cho1)
fc_cho = tf.layers.batch_normalization(fc_cho, training=is_training)
fc_cho = tf.nn.relu(fc_cho)
fc_cho= tf.nn.dropout(fc_cho,keep_prob)

fc_jung = tf.matmul(fc, W_jung1)
fc_jung = tf.layers.batch_normalization(fc_jung, training=is_training)
fc_jung = tf.nn.relu(fc_jung)
fc_jung= tf.nn.dropout(fc_jung,keep_prob)

fc_jong = tf.matmul(fc, W_jong1)
fc_jong = tf.layers.batch_normalization(fc_jong, training=is_training)
fc_jong = tf.nn.relu(fc_jong)
fc_jong= tf.nn.dropout(fc_jong,keep_prob)
print(fc)

Tensor("Reshape:0", shape=(?, 4096), dtype=float32)


In [11]:
W_cho2 = tf.Variable(tf.random_normal([256,19],stddev=0.01))
W_jung2 = tf.Variable(tf.random_normal([256,21],stddev=0.01))
W_jong2 = tf.Variable(tf.random_normal([256,28],stddev=0.01))

logit_cho = tf.matmul(fc_cho, W_cho2)
logit_jung = tf.matmul(fc_jung, W_jung2)
logit_jong = tf.matmul(fc_jong, W_jong2)

print(logit_cho)

Tensor("MatMul_3:0", shape=(?, 19), dtype=float32)


In [12]:
cross_entropy_cho = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_cho, labels=input_label_cho)
cross_entropy_jung = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jung, labels=input_label_jung)
cross_entropy_jong = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jong, labels=input_label_jong)

loss_cho = tf.reduce_mean(cross_entropy_cho)
loss_jung = tf.reduce_mean(cross_entropy_jung)
loss_jong = tf.reduce_mean(cross_entropy_jong)

total_loss = loss_cho + loss_jung + loss_jong

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(total_loss)
loss_summ=tf.summary.scalar("Loss",total_loss)
print(total_loss)

Tensor("add_1:0", shape=(), dtype=float32)


In [13]:
argmax_cho = tf.argmax(logit_cho,1)
argmax_jung = tf.argmax(logit_jung,1)
argmax_jong = tf.argmax(logit_jong,1)
print(argmax_cho)
prediction = tf.stack([argmax_cho,argmax_jung,argmax_jong],1)


Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [14]:
total_labels = tf.stack([input_label_cho, input_label_jung, input_label_jong],1)
equal_check = tf.equal(prediction,total_labels)
cast_equal = tf.cast(equal_check, tf.float32)
reduce_min = tf.reduce_min(cast_equal,1)

accuracy = tf.reduce_mean(reduce_min)
test_accuracy = tf.reduce_sum(reduce_min)
acc_summ=tf.summary.scalar("Accuracy",accuracy)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary = tf.summary.merge([loss_summ, acc_summ])
    train_writer = tf.summary.FileWriter("./logs/CNNV2/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV2/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy, total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary, feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
                
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/PE92/CNNV2/CNNV2_PE92.ckpt")
        
        acc_sum=0.0
        loss_sum=0.
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy,total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_cho)
        test_loss=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    
    print("Training is done.")
    sess.run(test_init_op)
    acc_sum=0.0
    loss_sum=0.0
    for epoch in range(total_test_batch):
        batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
        try:
            test_acc, test_loss=sess.run([test_accuracy,total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            loss_sum+=test_loss*len(batch_cho)
        
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
    
    test_loss=loss_sum/test_data_length
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))


    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 9.326351
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 8.783644
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 7.485042
Epoch :0 | step 300, training accuracy: 0.330000, train loss: 4.967968
Epoch :0 | step 400, training accuracy: 0.480000, train loss: 3.432831
Epoch :0 | step 500, training accuracy: 0.610000, train loss: 2.801615
Epoch :0 | step 600, training accuracy: 0.510000, train loss: 2.567747
Epoch :0 | step 700, training accuracy: 0.750000, train loss: 1.822445
Epoch :0 | step 800, training accuracy: 0.620000, train loss: 1.926883
Epoch :0 | step 900, training accuracy: 0.780000, train loss: 1.368045
Epoch :0 | step 1000, training accuracy: 0.730000, train loss: 1.320316
Epoch :0 | step 1100, training accuracy: 0.710000, train loss: 1.227501
Epoch :0 | step 1200, training accuracy: 0.810000, train loss: 1.155586
Epoch :0 | step 1300, training accuracy: 0.850000, t

In [ ]:
train_file_name = "Shuffled_with_alphabet_HanDB_Train.tfrecord"
test_file_name = "Shuffled_with_alphabet_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary3 = tf.summary.merge([loss_summ, acc_summ])
    train_writer = tf.summary.FileWriter("./logs/CNNV2/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV2/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
   
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy, total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary, feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/HanDB/CNNV2/CNNV2_HanDB.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy,total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                acc=test_acc/len(batch_cho)
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_cho)
        test_loss =loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
        
    
    print("Training is done.")
    acc_sum=0.0
    loss_sum=0.0
    sess.run(test_init_op)
    for epoch in range(total_test_batch):
        batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
        try:
            test_acc, test_loss=sess.run([test_accuracy,total_loss], feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            test_acc=test_acc/len(batch_cho)

        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
        loss_sum+=test_loss*len(batch_cho)
    test_loss=loss_sum/test_data_length
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))


    #Test Model
    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

In [ ]:
W2_cho2 = tf.Variable(tf.random_normal([256,19],stddev=0.01))
W2_jung2 = tf.Variable(tf.random_normal([256,21],stddev=0.01))
W2_jong2 = tf.Variable(tf.random_normal([256,28],stddev=0.01))

fc2_cho2 = tf.matmul(fc_cho, W_cho2)
bn2_cho2 = tf.layers.batch_normalization(fc2_cho2, training=is_training)
fc2_cho2 = tf.nn.relu(bn2_cho2)
fc2_cho2 = tf.nn.dropout(fc2_cho2, keep_prob)

fc2_jung2 = tf.matmul(fc_jung, W_jung2)
bn2_jung2 = tf.layers.batch_normalization(fc2_jung2, training=is_training)
fc2_jung2 = tf.nn.relu(bn2_jung2)
fc2_jung2 = tf.nn.dropout(fc2_jung2, keep_prob)

fc2_jong2 = tf.matmul(fc_jong, W_jong2)
bn2_jong2 = tf.layers.batch_normalization(fc2_jong2, training=is_training)
fc2_jong2 = tf.nn.relu(bn2_jong2)
fc2_jong2 = tf.nn.dropout(fc2_jong2, keep_prob)

In [ ]:
W_cho3 = tf.Variable(tf.random_normal([19,128],stddev=0.01))
W_jung3 = tf.Variable(tf.random_normal([21,128],stddev=0.01))
W_jong3 = tf.Variable(tf.random_normal([28,128],stddev=0.01))

fc3_cho2 = tf.matmul(fc2_cho2, W_cho3)
fc3_jung2 = tf.matmul(fc2_jung2, W_jung3)
fc3_jong2 = tf.matmul(fc2_jong2, W_jong3)

fc_total2 = fc3_cho2 + fc3_jung2 + fc3_jong2
bn_total2 = tf.layers.batch_normalization(fc_total2, training=is_training)
fc_total2 = tf.nn.relu(bn_total2)
fc_total2 = tf.nn.dropout(fc_total2, keep_prob)

W_total = tf.Variable(tf.random_normal([128,2350], stddev=0.01))
logit = tf.matmul(fc_total2, W_total)

In [ ]:
cross_entropy2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels)
loss = tf.reduce_mean(cross_entropy2)
update_ops2 = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops2):
    optimizer2 = tf.train.AdamOptimizer(0.0001).minimize(loss)
loss_summ2=tf.summary.scalar("Loss",loss)
        
correct_prediction2 = tf.equal(tf.argmax(logit,1), input_labels)
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
test_accuracy2 = tf.reduce_sum(tf.cast(correct_prediction2, tf.float32))
acc_summ2=tf.summary.scalar("Accuracy",accuracy2)

In [ ]:
def decode2(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [ ]:
batch_size=100
test_file_name = "Shuffled_PE92_Test.tfrecord"
train_file_name = "Shuffled_PE92_Train.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode2)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode2)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary3 = tf.summary.merge([loss_summ2, acc_summ2])
    train_writer = tf.summary.FileWriter("./logs/CNNV3/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV3/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels: batch_labels, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary3, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/PE92/CNNV3/CNNV3_PE92.ckpt")
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels = sess.run([test_image, test_label])
            try:
                test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_cho)
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    
    print("Training is done.")
    acc_sum=0.0
    loss_sum=0.0
    sess.run(test_init_op)
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
        loss_sum+=test_loss*len(batch_cho)
    test_loss=loss_sum/test_data_length
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))


    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

In [ ]:
train_file_name = "Shuffled_HanDB_Train.tfrecord"
test_file_name = "Shuffled_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode2)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode2)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label = train_iterator.get_next()
test_image, test_label = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [ ]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary4 = tf.summary.merge([loss_summ2, acc_summ2])
    train_writer = tf.summary.FileWriter("./logs/CNNV3/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV3/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)

    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(100):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([train_image, train_label])
            
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary4, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/HanDB/CNNV3/CNNV3_PE92.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels = sess.run([test_image, test_label])
            try:
                test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_cho)
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
    
    print("Training is done.")
    acc_sum=0.0
    loss_sum=0.0
    sess.run(test_init_op)
    for epoch in range(total_test_batch):
        batch_images, batch_labels = sess.run([test_image, test_label])
        try:
            test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0, is_training:False})
        except tf.errors.OutOfRangeError:
              print("Out of Data ierators")
        acc_sum+=test_acc
        loss_sum+=test_loss*len(batch_cho)
    test_loss=loss_sum/test_data_length
    test_acc = acc_sum/test_data_length
    print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))


    #Test Model
    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)